In [ ]:
# !pip install ibm-watsonx-ai==1.0.* | tail -n 1
# !pip install autoai-libs~=2.0 | tail -n 1
# !pip install scikit-learn==1.3.* | tail -n 1
# !pip install -U lale~=0.8.0 | tail -n 1
# !pip install snapml==1.14.* | tail -n 1

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='741a86e8-bf99-4d38-bc35-5f59970eaed3'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/7514b551-be66-42a4-b54b-a9dcea77512b/wml_data/4008eb09-7572-4516-a28d-d5658ec84166/data/automl',
        model_location='auto_ml/7514b551-be66-42a4-b54b-a9dcea77512b/wml_data/4008eb09-7572-4516-a28d-d5658ec84166/data/automl/model.zip',
        training_status='auto_ml/7514b551-be66-42a4-b54b-a9dcea77512b/wml_data/4008eb09-7572-4516-a28d-d5658ec84166/training-status.json'
    )
)

In [6]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='Amount(in rupees)',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='4bf71f09-dc20-4f1f-a6dc-d4c6d9bc660c',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[],
    feature_selector_mode='auto'
)

In [7]:
import os, ast
CPU_NUMBER = 4
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

In [8]:
api_key = 'ApiKey-7a8c1103-5cf7-4fed-bce1-8504d72c416c'

In [19]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url=experiment_metadata['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/2a0c52c5-766a-451c-a1bf-697416a0f320/predictions?version=2021-05-01']
)

KeyError: 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/2a0c52c5-766a-451c-a1bf-697416a0f320/predictions?version=2021-05-01'

In [10]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

if 'space_id' in experiment_metadata:
    client.set.default_space(experiment_metadata['space_id'])
else:
    client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(client)

Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

In [11]:
from autoai_libs.utils.outliers_mitigation import remove_outliers

df = remove_outliers(df, columns=[experiment_metadata['prediction_column']])

X_train, X_test, y_train, y_test = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=True)

NameError: name 'df' is not defined

In [12]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TA1
from snapml import SnapRandomForestRegressor
from sklearn.pipeline import make_pipeline

In [13]:
numpy_column_selector = NumpyColumnSelector(columns=[0, 1, 2, 3, 4, 5, 6])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], [], [], []],
)
numpy_replace_missing_values = NumpyReplaceMissingValues(
    filling_values=100001, missing_values=[]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=100001,
    filling_values_list=[
        100001, 100001, 100001, 100001, 100001, 100001, 100001,
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=100001,
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
ta2 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "Size", "location", "Floor", "Transaction", "Furnishing", "Bathroom",
        "Balcony",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 7),
    additional_col_count_to_keep=8,
    ptype="regression",
)
ta1 = TA1(
    fun=np.sqrt,
    name="sqrt",
    datatypes=["numeric"],
    feat_constraints=[
        autoai_libs.utils.fc_methods.is_non_negative,
        autoai_libs.utils.fc_methods.is_not_categorical,
    ],
    col_names=[
        "Size", "location", "Floor", "Transaction", "Furnishing", "Bathroom",
        "Balcony", "sum(location__Floor)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 7),
    additional_col_count_to_keep=8,
    ptype="regression",
)
snap_random_forest_regressor = SnapRandomForestRegressor(
    compress_trees=True,
    gpu_ids=np.array([0], dtype=np.uint32),
    max_depth=10,
    n_jobs=CPU_NUMBER,
    random_state=33,
)

In [14]:
pipeline = make_pipeline(
    numpy_column_selector,
    compress_strings,
    numpy_replace_missing_values,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
    ta2,
    fs1_0,
    ta1,
    fs1_1,
    snap_random_forest_regressor,
)

In [15]:
from sklearn.metrics import make_scorer

from autoai_libs.scorers.scorers import neg_root_mean_squared_error

scorer = make_scorer(neg_root_mean_squared_error)

In [16]:
pipeline.fit(X_train.values, y_train.values.ravel());

NameError: name 'X_train' is not defined

In [17]:
score = scorer(pipeline, X_test.values, y_test.values)
print(score)

NameError: name 'X_test' is not defined

In [ ]:
pipeline.predict(X_test.values[:5])

In [ ]:
model_metadata = {
    client.repository.ModelMetaNames.NAME: 'P4 - Pretrained AutoAI pipeline'
}

stored_model_details = client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

In [ ]:
stored_model_details